In [ ]:
import warnings
import logging
from research_notebooks.statarb_v2.performance_report import StatArbPerformanceReport
from datetime import datetime
import sys
import os
from dotenv import load_dotenv

logging.getLogger("asyncio").setLevel(logging.CRITICAL)
warnings.filterwarnings("ignore")

load_dotenv()
root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)

fetch_dbs = False
params = {
    "mongo_uri": os.getenv("MONGO_URI", ""),
    "database": "quants_lab",
    "from_timestamp": datetime(2025, 3, 10, 0, 0, 0).timestamp(),
    "to_timestamp": datetime(2025, 3, 16, 23, 59, 59).timestamp(),
    "root_path": root_path,
    "backend_host": os.getenv("BACKEND_API_SERVER"),
    "backend_user": os.getenv("BACKEND_API_SERVER_USER", "root"),
    "backend_data_path": os.getenv("BACKEND_API_SERVER_DATA_PATH", "deploy/bots/archived"),
}
performance_report = StatArbPerformanceReport(**params)
await performance_report.initialize()
if fetch_dbs:
    performance_report.fetch_dbs(params["root_path"],
                                 params["backend_host"],
                                 params["backend_user"],
                                 params["backend_data_path"])
await performance_report.load_data()
trading_sessions = await performance_report.build_trading_sessions()
print(performance_report.summary)

In [ ]:
performance_report.trading_sessions[5].short_performance_fig

In [ ]:
performance_report.create_cointegration_heatmap()

In [ ]:
performance_report.create_gantt_chart()

In [ ]:
performance_report.plot_volume_treemap()

In [ ]:
import pandas as pd

coint_data = pd.DataFrame([session.coint_info for session in performance_report.trading_sessions])
coint_data.dropna(subset=["base_asset", "quote_asset"], inplace=True)
coint_data


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Copy DataFrame
df = coint_data.copy()

# Convert timestamp column to numeric
df["timestamp"] = pd.to_numeric(df["timestamp"])

# Keep only the last timestamp per (base_asset, quote_asset) pair
df_filtered = df.loc[df.groupby(["base_asset", "quote_asset"])["timestamp"].idxmax()]

# Reset index (optional)
df_filtered = df_filtered[(df_filtered["base_asset"].isin(performance_report.trading_pairs)) &
                          (df_filtered["quote_asset"].isin(performance_report.trading_pairs))]

# Create a symmetric matrix
cointegration_matrix = df_filtered.pivot(index="base_asset", columns="quote_asset", values="coint_value")
cointegration_matrix = cointegration_matrix.combine_first(cointegration_matrix.T)

# Fill diagonal with 1.0 (or NaN if preferred)
for asset in set(df_filtered["base_asset"]).union(df_filtered["quote_asset"]):
    cointegration_matrix.loc[asset, asset] = 1.0

# Get median value for white midpoint
median_value = np.nanmedian(cointegration_matrix.values)

np.fill_diagonal(cointegration_matrix.values, np.nan)

# Define custom colorscale
custom_colorscale = [
    [0.0, "green"],   # Low values (close to 0) → Green
    [0.5, "white"],   # Median value → White
    [1.0, "red"]      # High values → Red
]

# Replace NaNs in annotations with an empty string
annotations = np.where(pd.isna(cointegration_matrix.values), "", np.round(cointegration_matrix.values, 2))

# Create Heatmap with Annotations
fig = go.Figure(data=go.Heatmap(
    z=cointegration_matrix.values,
    x=cointegration_matrix.columns,
    y=cointegration_matrix.index,
    colorscale=custom_colorscale,
    text=annotations,  # Annotate with cointegration values
    hoverinfo="text",
    texttemplate="%{text}",  # Display text in boxes
    zmid=median_value,  # Set the white midpoint for balance
))

# Layout Adjustments
fig.update_layout(
    title=f"Cointegration Analysis of Traded Markets ({len(performance_report.trading_pairs)} Selected Pairs)",
    xaxis_title="Long Asset",
    yaxis_title="Short Asset",
    autosize=False,
    width=1200,
    height=1000,
    font=dict(size=12)
)

fig.write_image("correlation_heatmap.jpg", format="jpg", scale=3)
